# Élections municipales 2020 Toulouse

## Découpage des bureaux de vote

https://data.toulouse-metropole.fr/explore/dataset/elections-2020-decoupage-bureaux-de-vote-toulouse/information/

In [1]:
import folium, json
import pandas

m = folium.Map(location=[43.60437,1.44227], zoom_start=12) # centre de la carte

dataset_decoupage_bdv = 'elections-2020-decoupage-bureaux-de-vote-toulouse.geojson'

folium.GeoJson(dataset_decoupage_bdv,
        name="découpage bdv",
        popup = folium.features.GeoJsonPopup(fields=['nom', 'bv2020'], aliases = ["Lieu de vote : ", "Numéro bureau : "], labels = True)
       ).add_to(m)

# ajout une icône pour controler l'affichage
folium.LayerControl().add_to(m)

folium.__version__ # 0.10.1 : pas de popup geojson

m

## Carte choroplèthe

https://fr.wikipedia.org/wiki/Carte_choropl%C3%A8the

Nous allons affecter une couleur à chaque zone en fonction des résultats aux élections.

On charge d'abord les données :

In [2]:
import pandas as pd

resultats = pd.read_csv("elections-municipales-et-communautaires-2020-2eme-tour-toulouse-resultats.csv", sep=";")

resultats.head(4)

,Numéro,Type election,Annee,Type enregistrement,Numéro tour,Code departement,Code commune,Numéro bdv,Indicatif,Inscrits,...,Votants,Votants emargement,Blancs,Nuls,Exprimes,Nombre de listes,Archipel Citoyen,Nombre de voix de la liste : Archipel Citoyen,Aimer Toulouse avec Jean-Luc Moudenc,Nombre de voix de la liste : Aimer Toulouse
0,11,MN,2020,V,2,31,555,11,I,1025,...,593,593,12,2,579,2,14,273,10,306
1,34,MN,2020,V,2,31,555,34,I,696,...,359,359,1,2,356,2,14,154,10,202
2,35,MN,2020,V,2,31,555,35,I,822,...,482,482,6,3,473,2,14,263,10,210
3,38,MN,2020,V,2,31,555,38,I,869,...,435,435,5,8,422,2,14,226,10,196


On s'intéresse plus particulièrement au numéro du bureau de vote, les inscrits, les votants, les votes pour AC et pour AT :

In [3]:
resultats[["Numéro bdv",
           "Inscrits", 
           "Votants", 
           "Exprimes",
           "Nombre de voix de la liste : Archipel Citoyen",
           "Nombre de voix de la liste : Aimer Toulouse"]].head()

,Numéro bdv,Inscrits,Votants,Exprimes,Nombre de voix de la liste : Archipel Citoyen,Nombre de voix de la liste : Aimer Toulouse
0,11,1025,593,579,273,306
1,34,696,359,356,154,202
2,35,822,482,473,263,210
3,38,869,435,422,226,196
4,47,962,542,530,254,276


On rajoute à notre DataFrame, une colonne donnant le pourcentage de voix en faveur de chacune des listes :

In [4]:
resultats["pourcentageAT"]=round((resultats["Nombre de voix de la liste : Aimer Toulouse"]/resultats["Exprimes"])*100,2)
resultats["pourcentageAC"] = round(100 - resultats["pourcentageAT"],2)
resultats.head()

,Numéro,Type election,Annee,Type enregistrement,Numéro tour,Code departement,Code commune,Numéro bdv,Indicatif,Inscrits,...,Blancs,Nuls,Exprimes,Nombre de listes,Archipel Citoyen,Nombre de voix de la liste : Archipel Citoyen,Aimer Toulouse avec Jean-Luc Moudenc,Nombre de voix de la liste : Aimer Toulouse,pourcentageAT,pourcentageAC
0,11,MN,2020,V,2,31,555,11,I,1025,...,12,2,579,2,14,273,10,306,52.85,47.15
1,34,MN,2020,V,2,31,555,34,I,696,...,1,2,356,2,14,154,10,202,56.74,43.26
2,35,MN,2020,V,2,31,555,35,I,822,...,6,3,473,2,14,263,10,210,44.40,55.60
3,38,MN,2020,V,2,31,555,38,I,869,...,5,8,422,2,14,226,10,196,46.45,53.55
4,47,MN,2020,V,2,31,555,47,I,962,...,10,2,530,2,14,254,10,276,52.08,47.92


Au passage, dans quel bureau de vote la liste AT a-t-elle eu le plus fort score ? Et le plus faible ?

In [5]:
index_min = resultats["pourcentageAT"].idxmin()
val_min = resultats["pourcentageAT"].min()

index_max = resultats["pourcentageAT"].idxmax()
val_max = resultats["pourcentageAT"].max()

bdv_min_AT = resultats.iloc[index_min]["Numéro bdv"]
exprimes_min = resultats.iloc[index_min]["Exprimes"]

bdv_max_AT = resultats.iloc[index_max]["Numéro bdv"]
exprimes_max = resultats.iloc[index_max]["Exprimes"]


print("Min de {} %, pour le bureau {} avec {} exprimés".format(val_min,bdv_min_AT, exprimes_min))
print("Max de {} %, pour le bureau {} avec {} exprimés".format(val_max,bdv_max_AT, exprimes_max))

Min de 25.0 %, pour le bureau 71 avec 16 exprimés
Max de 83.7 %, pour le bureau 63 avec 460 exprimés


Et où sont alors ces deux extrêmes ? Nous allons définir deux styles différents selon les valeurs avec une fonction `lambda`.

In [6]:
n = folium.Map(location=[43.60437,1.44227], zoom_start=14) # centre de la carte

dataset_decoupage_bdv = 'elections-2020-decoupage-bureaux-de-vote-toulouse.geojson'

folium.GeoJson(dataset_decoupage_bdv,
        name="découpage bdv",
        style_function = lambda feature: {
        'fillColor': 'darkblue' if feature['properties']['bv2020'] == bdv_max_AT 
                        else ('green' if feature['properties']['bv2020'] == bdv_min_AT else 'white'),
        'color': 'blue',
        'weight': 2,
        'dashArray': '5, 5'
        },
        popup = folium.features.GeoJsonPopup(fields=['nom', 'bv2020'], aliases = ["Lieu de vote : ", "Numéro bureau : "], labels = True)

       ).add_to(n)
n

## Carte choroplèthe

Autant faire directement une carte choroplèthe où la couleur de la zone va dépendre du score de chaque liste

Nous allons créer un DataFrame contenant deux variables :

* le numéro de bureau de vote
* le pourcentage de votes pour la liste AT

In [7]:
df = pandas.DataFrame({
    "Numéro du bureau de vote" : resultats["Numéro bdv"],
    "Votes pour AT" : resultats["pourcentageAT"],
    "Votes pour AC" : resultats["pourcentageAC"]
})

df


,Numéro du bureau de vote,Votes pour AT,Votes pour AC
0,11,52.85,47.15
1,34,56.74,43.26
2,35,44.40,55.60
3,38,46.45,53.55
4,47,52.08,47.92
...,...,...,...
258,177,47.14,52.86
259,179,55.18,44.82
260,182,47.12,52.88
261,186,50.97,49.03


Pour relier ces informations à la carte, nous devons définir les éléments suivants :

* geo_data : les contours au format GeoJSON
* key_on : l'item dans le GeoJSON avec lequel nous ferons la jointure, ici le numéro de bureau de vote.
* data : le DataFrame dans lequel nous avons les informationss tatistiques
* columns : les deux colonnes à prendre
 * clé de jointure : le numéro de bureau de vote
 * mesure statistique : le % de votants pour AT
* fill_color : la palette de couleurs à utiliser.

On peut alors créer la carte choroplèthe.

In [8]:
toulouse = folium.Map(location=[43.60437,1.44227], zoom_start=14) # centre de la carte

choropleth = folium.Choropleth(
    geo_data='elections-2020-decoupage-bureaux-de-vote-toulouse.geojson',
    name='choropleth',
    data=df,
    columns=["Numéro du bureau de vote",'Votes pour AT'],
    key_on='feature.properties.bv2020',
    bins = [0,20,30,40,50,60,70,80,100],
    fill_color='RdBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Votes pour la liste "Aimer Toulouse" (%)',
    highlight=True,
).add_to(toulouse)

toulouse


Ce serait intéressante que par survol avec la souris d'une zone, nous ayons le score de chaque liste.

Nous allons utiliser `folium.features.GeoJsonTooltip`.

Dans le tooltip, on souhaite afficher le pourcentage de votants en faveur de la liste AT et AC.

Il va donc falloir ajouter les données dans le fichier GeoJson. 

Tout d'abord on le charge et observe sa structure. Nous allons ajouter la donnée dans le même dictionnaire que là où se trouve la clé `bv2020`. (Identifiant du bureau de vote): 

In [9]:
import json

geo = json.load(open('elections-2020-decoupage-bureaux-de-vote-toulouse.geojson'))

geo.keys(), geo["features"][0]['properties']

(dict_keys(['type', 'features']),
 {'nom': 'CAPITOLE SALLE DES COMMISSIONS',
  'adresse': 'HOTEL DE VILLE',
  'uniq_bdv': '0001',
  'bv2020': '1',
  'geo_point': [43.6032242795, 1.44410768632]})

Les scores sont dans le DataFrame suivant :

In [10]:
df.head()

,Numéro du bureau de vote,Votes pour AT,Votes pour AC
0,11,52.85,47.15
1,34,56.74,43.26
2,35,44.40,55.60
3,38,46.45,53.55
4,47,52.08,47.92


On va fixer la colonne "Numéro du bureau de vote" comme nouvel index.

Ce sera plus simple (pour moi !) afin d'accéder au bon pourcentage de votes pour AT ou AC en fonction du numéro du bureau de vote.

In [11]:
new_df=df.set_index('Numéro du bureau de vote')

In [12]:
new_df.loc['11','Votes pour AT'] # donne 52.85 pour le bureau 11

52.85

In [13]:
new_df.index

Index(['11', '34', '35', '38', '47', '49', '50', '61', '69', '74',
       ...
       '150', '157', '160', '169', '171', '177', '179', '182', '186', '192A'],
      dtype='object', name='Numéro du bureau de vote', length=263)

On ajoute la donnée dans le dictionnaire `properties` du geojson au bon endroit... Pour éviter de tout mélanger, on prendra comme clé de jointure le numéro du bureau de vote.

In [14]:
for num_bdv in new_df.index:
    # double-boucle for, c'est moche...
    for i in range(len(geo["features"])):
        if geo["features"][i]['properties']['bv2020'] == num_bdv:
            geo["features"][i]['properties']['pourcentageAT']=new_df.loc[num_bdv, "Votes pour AT"]
            geo["features"][i]['properties']['pourcentageAC']=new_df.loc[num_bdv, "Votes pour AC"]

               
#vérification
print(geo["features"][0]['properties'], new_df.loc['1', 'Votes pour AT'])

{'nom': 'CAPITOLE SALLE DES COMMISSIONS', 'adresse': 'HOTEL DE VILLE', 'uniq_bdv': '0001', 'bv2020': '1', 'geo_point': [43.6032242795, 1.44410768632], 'pourcentageAT': 56.99, 'pourcentageAC': 43.01} 56.99


On peut enfin regénérer une carte avec le nouveau fichier geoJson puis on y ajoutera les `GeoJsonTooltip` :

In [15]:
toulouse = folium.Map(location=[43.60437,1.44227], zoom_start=14) # centre de la carte

choropleth=folium.Choropleth(
    geo_data=geo,
    name='choropleth',
    data=df,
    columns=["Numéro du bureau de vote",'Votes pour AT'],
    key_on='feature.properties.bv2020',
    bins = [0,20,30,40,50,60,70,80,100],
    fill_color='RdBu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Votes pour la liste "Aimer Toulouse" (%)',
    highlight=True,
).add_to(toulouse)

In [17]:
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['nom','bv2020','pourcentageAT', 'pourcentageAC'], aliases = ['Lieu de vote', 'Numéro bureau','% pour Aimer Toulouse', '% pour Archipel Citoyen'])
)

toulouse.save('map.html')